In [14]:
pip install telethon python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
import logging


In [18]:
load_dotenv()

API_ID = int(os.getenv('TELE_API_ID'))
API_HASH = os.getenv('TELE_API_HASH')
PHONE_NUMBER = os.getenv('TELE_PHONE_NUMBER')



Setup Logging

In [ ]:
logging.basicConfig(
    filename='telegram_scraping_new.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
